In [2]:
%load_ext autoreload
%autoreload 2

import os
import neuroglancer
import numpy as np
import networkx as nx
import operator
import sys
import itertools
from pathlib import Path

import zarr
from funlib.show.neuroglancer import add_layer
from funlib.persistence.graphs import SQLiteGraphDataBase, PgSQLGraphDatabase
from funlib.persistence import open_ds

In [2]:
import funlib.persistence
funlib.persistence.__file__

'/home/anton/miniforge3/envs/segmentation/lib/python3.9/site-packages/funlib/persistence/__init__.py'

In [23]:
rag = SQLiteGraphDataBase(Path("test.sqlite"),
  ["center_z", "center_y", "center_x"],
  mode="r",
  edge_attrs={"merge_score": float, "agglomerated": bool}
)
#rag = SQLiteGraphDataBase(Path("../../data/oblique.sqlite"),
#  ["position_z", "position_y", "position_x"],
#  mode="r"
#)

In [24]:
edges = rag.read_edges()

In [27]:
merge_scores = [edge["merge_score"] for edge in edges] 
values, counts = np.unique(np.array(merge_scores), return_counts=True)

In [65]:
num_edges = sum(counts)
tot_count = 0
threshold = 0.1
for i, val in enumerate(values):
  if val < threshold:
    tot_count += counts[i]
print(f"{(tot_count / num_edges) * 100:.2f}%")

43.30%


In [29]:
num_edges

2253689

In [17]:
len(merge_scores)

254

In [11]:
rag2 = SQLiteGraphDataBase(
  Path("../../data/blockwise.zarr/repost/affs_200000_FalseNorm_FalseBoundaryMask40_15MinSeedDist_0FragFilter/rag.db"),
  position_attributes=["position_z", "position_y", "position_x"],
  edge_attrs={"merge_score": float, "agglomerated": bool},
  edges_table="edges_hist_quant_75"
)

In [18]:
edges2 = rag2.read_edges()

In [20]:
merge_scores2 = [edge["merge_score"] for edge in edges2]
np.unique(np.array(merge_scores2), return_counts=True)

(array([0.00585938, 0.00976562, 0.01367188, 0.01757812, 0.02148438,
        0.02539062, 0.02929688, 0.03320312, 0.03710938, 0.04101562,
        0.04492188, 0.04882812, 0.05273438, 0.05664062, 0.06054688,
        0.06445312, 0.06835938, 0.07226562, 0.07617188, 0.08007812,
        0.08398438, 0.08789062, 0.09179688, 0.09570312, 0.09960938,
        0.10351562, 0.10742188, 0.11132812, 0.11523438, 0.11914062,
        0.12304688, 0.12695312, 0.13085938, 0.13476562, 0.13867188,
        0.14257812, 0.14648438, 0.15039062, 0.15429688, 0.15820312,
        0.16210938, 0.16601562, 0.16992188, 0.17382812, 0.17773438,
        0.18164062, 0.18554688, 0.18945312, 0.19335938, 0.19726562,
        0.20117188, 0.20507812, 0.20898438, 0.21289062, 0.21679688,
        0.22070312, 0.22460938, 0.22851562, 0.23242188, 0.23632812,
        0.24023438, 0.24414062, 0.24804688, 0.25195312, 0.25585938,
        0.25976562, 0.26367188, 0.26757812, 0.27148438, 0.27539062,
        0.27929688, 0.28320312, 0.28710938, 0.29

In [2]:
psql = PgSQLGraphDatabase(
  position_attributes=["center_z", "center_y", "center_x"],
  edge_attrs={"merge_score": float, "agglomerated": bool},
  db_name="anton_testing_2",
  db_host="nnpostgres.pods.tacc.tapis.io",
  db_user="nnpostgres",
  db_password="0UXMqUOwMg6pMdwVJo4wuh3Xj2qgt1",
  db_port="443",
  mode="w"
)

: 

In [3]:
psql = PgSQLGraphDatabase(
  position_attributes=["center_z", "center_y", "center_x"],
  edge_attrs={"merge_score": float, "agglomerated": bool},
  db_name="anton_testing_2",
  db_host="khlabgpu.clm.utexas.edu",
  db_user="anton",
  db_password="password",
  db_port="5433",
  mode="w"
)

['merge_score', 'agglomerated']


In [7]:
psql.read_edges()

[]

In [8]:
del psql

In [9]:
G = rag.read_graph()

In [15]:
G.nodes[0]["position_z"]

600.0

In [16]:
for node in G.nodes():
  G.nodes[node]["center_x"] = G.nodes[node]["position_x"]
  G.nodes[node]["center_y"] = G.nodes[node]["position_y"]
  G.nodes[node]["center_z"] = G.nodes[node]["position_z"]

In [17]:
psql.write_graph(G)

In [18]:
psql.read_edges()

[{'u': 0, 'v': 1, 'merge_score': None, 'agglomerated': None},
 {'u': 0, 'v': 530, 'merge_score': None, 'agglomerated': None},
 {'u': 1, 'v': 2, 'merge_score': None, 'agglomerated': None},
 {'u': 2, 'v': 3, 'merge_score': None, 'agglomerated': None},
 {'u': 3, 'v': 4, 'merge_score': None, 'agglomerated': None},
 {'u': 4, 'v': 5, 'merge_score': None, 'agglomerated': None},
 {'u': 5, 'v': 6, 'merge_score': None, 'agglomerated': None},
 {'u': 6, 'v': 7, 'merge_score': None, 'agglomerated': None},
 {'u': 7, 'v': 8, 'merge_score': None, 'agglomerated': None},
 {'u': 8, 'v': 9, 'merge_score': None, 'agglomerated': None},
 {'u': 9, 'v': 10, 'merge_score': None, 'agglomerated': None},
 {'u': 10, 'v': 11, 'merge_score': None, 'agglomerated': None},
 {'u': 11, 'v': 12, 'merge_score': None, 'agglomerated': None},
 {'u': 12, 'v': 13, 'merge_score': None, 'agglomerated': None},
 {'u': 13, 'v': 14, 'merge_score': None, 'agglomerated': None},
 {'u': 14, 'v': 15, 'merge_score': None, 'agglomerated': No

In [6]:
edges = rag.read_edges()

In [17]:
edges[105]["agglomerated"]

1

In [4]:
G = rag.read_graph()

In [5]:
G[0]

KeyError: 0

In [4]:
edges = rag.read_graph().edges(data=True)

TypeError: 'ItemsView' object is not subscriptable

In [5]:
for e in G.edges.items():
  print(e)
  break

((1, 1306369), {'merge_score': 0.115234375})


In [11]:
G = rag.read_graph()

In [8]:
rag = SQLiteGraphDataBase(Path("test2.sqlite"),
  ["position_z", "position_y", "position_x"],
  mode="w"
)

In [9]:
rag.write_graph(G)

IntegrityError: datatype mismatch

In [ ]:
rag.re